In [1]:
import torchtext
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from tqdm import tqdm_notebook as tqdm
import json
import torch
from transformers import *
from transformers import AdamW
import torch.utils.data as Data
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext.vocab as Vocab
import pickle as pk
import copy
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
from IPython.display import display,HTML
import os
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pack_sequence
from torch.nn import CrossEntropyLoss, MSELoss
from torchcrf import CRF
from sklearn import metrics
import joblib
import math
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import argparse
import glob
import json
import logging
import unicodedata
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook as tqdm
import torch.utils.data as Data
import jieba
import jieba.posseg as pseg
import numpy as np
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO 
)
ARG=collections.namedtuple('ARG',['train_batch_size',
 'eval_batch_size',
 'weight_decay',
 'learning_rate',
 'adam_epsilon',
 'num_train_epochs',
 'warmup_steps',
 'gradient_accumulation_steps',
 'save_steps',
 'max_grad_norm',
 'model_name_or_path',
 'output_dir',
 'seed',
 'device',
 'n_gpu',
 'max_steps',
 'output_mode',
'fp16_opt_level',
'fp16',
'card_list'])

unable to import 'smart_open.gcs', disabling that module


In [2]:
do_lower_case=True
max_len=256
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# bert_dir="./bert-pytorch-chinese/"
# vocab="bert-base-chinese-vocab.txt"
# config_file="bert_config.json"
bert_dir="./roberta-zh-wwm-pytorch/"
vocab="vocab.txt"
config_file="bert_config.json"
tokenizer=BertTokenizer.from_pretrained(os.path.join(bert_dir,vocab),do_lower_case=do_lower_case)


05/18/2020 23:05:49 - INFO - transformers.tokenization_utils -   Model name './roberta-zh-wwm-pytorch/vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming './roberta-zh-wwm-pytorch/vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
05/18/2020 23:05:49 - WARNING - transformers.tokenization_utils -   Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
05/18/2020 23:05:49 - 

In [3]:
def convert_text_to_ids(tokenizer, text, max_len=100):
    if isinstance(text,str):
        output=tokenizer.encode_plus(text,max_length=max_len,pad_to_max_length=True,return_tensors="pt")
        input_ids=output["input_ids"].squeeze(0)
        token_type_ids=output["token_type_ids"].squeeze(0)
        attention_mask=output["attention_mask"].squeeze(0)
    elif isinstance(text,list):
        input_ids,token_type_ids,attention_mask=[],[],[]
        for e in text:
            output=tokenizer.encode_plus(e,max_length=max_len,pad_to_max_length=True,return_tensors="pt")
            input_ids.append(output["input_ids"].squeeze(0))
            token_type_ids.append(output["token_type_ids"].squeeze(0))
            attention_mask.append(output["attention_mask"].squeeze(0))
    else:
        raise Exception('type error')
    return torch.stack(input_ids).long(),torch.stack(token_type_ids).long(),torch.stack(attention_mask).long()        
class RelDataset(Data.Dataset):
    def __init__(self,examples):
        self.input_ids=torch.stack([e['input_ids'] for e in examples]).long()
        self.token_type_ids=torch.stack([e['token_type_ids'] for e in examples]).long()
        self.attention_mask=torch.stack([e['attention_mask'] for e in examples]).long()
        self.rel_label=torch.stack([e['rel_label'] for e in examples]).long()
        self.postag=torch.stack([e['postag'] for e in examples]).long()
        self.feature=torch.stack([e['feature'] for e in examples]).float()
        self.token2docs=[e["token2doc"] for e in examples]
    def __len__(self):
        return self.input_ids.shape[0]
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.token_type_ids[idx],\
               self.rel_label[idx],self.postag[idx],self.feature[idx],self.token2docs[idx]  
class NerDataset(Data.Dataset):
    def __init__(self,examples):
        self.input_ids=torch.stack([e['input_ids'] for e in examples]).long()
        self.token_type_ids=torch.stack([e['token_type_ids'] for e in examples]).long()
        self.attention_mask=torch.stack([e['attention_mask'] for e in examples]).long()
        self.rel_label=torch.stack([e['rel_label'] for e in examples]).long()
        self.labels=torch.stack([e['labels'] for e in examples]).long()
        self.postag=torch.stack([e['postag'] for e in examples]).long()
        self.feature=torch.stack([e['feature'] for e in examples]).float()
        self.token2docs=[e["token2doc"] for e in examples]
    def __len__(self):
        return self.input_ids.shape[0]
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.token_type_ids[idx],\
               self.rel_label[idx],self.labels[idx],self.postag[idx],self.feature[idx],self.token2docs[idx]  
import unicodedata
def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False
def is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
        (cp >= 0x4E00 and cp <= 0x9FFF)
        or (cp >= 0x3400 and cp <= 0x4DBF)  #
        or (cp >= 0x20000 and cp <= 0x2A6DF)  #
        or (cp >= 0x2A700 and cp <= 0x2B73F)  #
        or (cp >= 0x2B740 and cp <= 0x2B81F)  #
        or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
        or (cp >= 0xF900 and cp <= 0xFAFF)
        or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False
def is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def build_tfidf_svd_matrix(texts,n_output,tfidf_vec=None,svd=None):
    corpus=[]
    for text in tqdm(texts):
#         print(text)
        words=word_segment(str(text['text']))
#         print(words)
        use_words=[]
        for word in words:
            if word in stop_words:
                continue
            use_words.append(word)
#         print(use_words)
        corpus.append(" ".join(use_words))
    print(len(corpus))
    print(corpus[0])
    if tfidf_vec is None:
        tfidf_vec=TfidfVectorizer()
        tfidf_matrix=tfidf_vec.fit_transform(corpus)
    else:
        tfidf_matrix=tfidf_vec.transform(corpus)
    if svd is None:
        svd=TruncatedSVD(n_components=n_output,n_iter=7,random_state=42)
        tf_idf_svd=svd.fit_transform(tfidf_matrix)
    else:
        tf_idf_svd=svd.transform(tfidf_matrix)
    return tf_idf_svd,tfidf_vec,svd
def word_segment(sentence):
    words=jieba.cut(sentence)
    return ",".join(words).split(",")
stop_words=set()
def load_stopwords():
    with open("./middle_data/stopwords.txt","r",encoding="UTF-8") as r:
        for line in r.readlines():
            stop_words.add(line.strip())
load_stopwords()
def remove_stopwords(word_list):
    res=[]
    for word in word_lists:
        if word not in stop_words:
            res.append(word)
    return ' '.join(res)
def clean_text(string):
    return string.replace(' ', '').replace('\n', '').replace('\u3000', '')
def _convert_example_to_record(example,
                               tokenizer):
    if example.__contains__('spo_list'):
        spo_list = example['spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels_list=[[0
              for i in range(len(tokens))] for i in range(len(id2rels))]
    rel_labels=[]
    for spo in spo_list:
        for spo_object in spo['object'].keys():
            if not spo['predicate'] in special_rels:
                rel_label=rels2id[spo["predicate"]]
            else:
                rel_label=rels2id[spo["predicate"]+"_"+spo_object] 
            labels = labels_list[rel_label] #复杂类的不同part还是不会被归到一个，以后再讲
            label_subject = label2ids['B-SUB']
            label_object = label2ids['B-OBJ']
            subject_sub_tokens = tokenizer.tokenize(spo['subject'])
            object_sub_tokens = tokenizer.tokenize(spo['object'][
                spo_object])
            forbidden_index = None
            if len(subject_sub_tokens) > len(object_sub_tokens):
                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        labels[index]=label_subject
                        for i in range(len(subject_sub_tokens) - 1):
                            labels[index + i + 1]=label_subject+1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        if forbidden_index is None:
                            labels[index]=label_object
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1]=label_object+1
                            break
                        # check if labeled already
                        elif index < forbidden_index or index >= forbidden_index + len(
                                subject_sub_tokens):
                            labels[index]=label_object
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1]=label_object+1
                            break

            else:
                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        labels[index]=label_object
                        for i in range(len(object_sub_tokens) - 1):
                            labels[index + i + 1]=label_object+1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        if forbidden_index is None:
                            labels[index]=label_subject
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1]=label_subject+1
                            break
                        elif index < forbidden_index or index >= forbidden_index + len(
                                object_sub_tokens):
                            labels[index]=label_subject
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1]=label_subject+1
                            break
            labels_list[rel_label]=labels
            if rel_label not in rel_labels:
                rel_labels.append(rel_label)

    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels
def _convert_example_to_cls_record(example,
                               tokenizer):
    if example.__contains__('spo_list'):
        spo_list = example['spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels_list=[]
    rel_labels=[]
    for spo in spo_list:
        for spo_object in spo['object'].keys():
            labels = [0
                  for i in range(len(tokens))]  # initialize tag
            if not spo['predicate'] in special_rels:
                rel_label=rels2id[spo["predicate"]]
            else:
                rel_label=rels2id[spo["predicate"]+"_"+spo_object] 
            rel_labels.append(rel_label)

    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,rel_labels
def create_cls_example(tokens,rel,token2doc,postag,tfidf_svd,tokenizer):
        if len(tokens)>max_len-2:
                tokens=tokens[:(max_len-2)]
                token2doc=[e[:(max_len-2)] if ind<2 else e for ind,e in enumerate(token2doc)]
        tag=[postag[token2doc[0][idx]] for idx in range(len(tokens))]
        tag=[pos2id_BIO[tokenizer.cls_token]]+tag+[pos2id_BIO[tokenizer.sep_token]]
        full_tokens=[tokenizer.cls_token]+tokens+[tokenizer.sep_token]
        token_type_ids=[0]*len(full_tokens)
        attention_mask=[1]*len(token_type_ids)
        cur_len=len(full_tokens)
        if cur_len<max_len:
            full_tokens+=[tokenizer.pad_token]*(max_len-cur_len)
            attention_mask+=[0]*(max_len-cur_len)
            token_type_ids+=[0]*(max_len-cur_len)
            tag+=[pos2id_BIO[tokenizer.pad_token]]*(max_len-cur_len)
        full_ids=tokenizer.convert_tokens_to_ids(full_tokens)
        if len(rel)>0:
            example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                    "attention_mask":torch.tensor(attention_mask,dtype=torch.long),
                    "rel_label":(F.one_hot(torch.tensor(rel),len(id2rels)).sum(dim=0)!=0).long(),
                     "postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),"token2doc":token2doc}
        else:
            example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                    "attention_mask":torch.tensor(attention_mask,dtype=torch.long),
                    "rel_label":(torch.zeros(len(id2rels))).long(),\
                     "postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),"token2doc":token2doc}  
        return example
def create_example(tokens,rel,labels,token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc):
        tag=[postag[token2doc[0][idx]] for idx in range(len(tokens))]
        tag=[pos2id_BIO[tokenizer.cls_token]]+tag+[pos2id_BIO[tokenizer.sep_token]]
        rel_tag=[rel_postag[rel_token2doc[idx]] for idx in range(len(rel_text))]
        rel_tag=rel_tag+[pos2id_BIO[tokenizer.sep_token]]
        tag=tag+rel_tag
        second_token=rel_text
        full_tokens=[[tokenizer.cls_token]+tokens+[tokenizer.sep_token],second_token+[tokenizer.sep_token]]
        full_labels=[[label2ids[tokenizer.cls_token]]+labels+[label2ids[tokenizer.sep_token]],[label2ids["[category]"]]*len(second_token)+[label2ids[tokenizer.sep_token]]]
        token_type_ids=[0]*len(full_tokens[0])+[1]*len(full_tokens[1])
        attention_mask=[1]*len(token_type_ids)
        full_tokens=full_tokens[0]+full_tokens[1]
        full_labels=full_labels[0]+full_labels[1]
        cur_len=len(full_labels)
        if cur_len<max_len:
            full_tokens+=[tokenizer.pad_token]*(max_len-cur_len)
            full_labels+=[label2ids[tokenizer.pad_token]]*(max_len-cur_len)
            attention_mask+=[0]*(max_len-cur_len)
            token_type_ids+=[0]*(max_len-cur_len)
            tag+=[pos2id_BIO[tokenizer.pad_token]]*(max_len-cur_len)
        full_ids=tokenizer.convert_tokens_to_ids(full_tokens)
        example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                "attention_mask":torch.tensor(attention_mask,dtype=torch.long),"labels":torch.tensor(full_labels,dtype=torch.long),
                "rel_label":F.one_hot(torch.tensor(rel),num_classes=len(id2rels)),"postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),\
                 "token2doc":token2doc}
        return example
def index_token(text_raw):
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index
def find_subs_obs(token_label_pred,text):
    subs=[]
    sub_offsets=[]
    obs=[]
    ob_offsets=[]
    index=0
    while index<token_label_pred.shape[0]:
        if token_label_pred[index]==3:
            subs.append(text[index])
            index+=1
            while(index<token_label_pred.shape[0] and (token_label_pred[index]==4 or token_label_pred[index]==3)):
                subs[-1]+=(text[index])
                index+=1
            sub_offsets.append([index-len(subs[-1]),index])
            index-=1
        if index<token_label_pred.shape[0] and token_label_pred[index]==1:
            obs.append(text[index])
            index+=1
            while(index<token_label_pred.shape[0] and (token_label_pred[index]==2 or token_label_pred[index]==1)):
                obs[-1]+=(text[index])
                index+=1
            ob_offsets.append([index-len(obs[-1]),index])
            index-=1
        index+=1
    if len(subs)==0:
        subs.append('')
        sub_offsets.append([0,0])
    if len(obs)==0:
        obs.append('')
        ob_offsets.append([0,0])
    return subs,obs,sub_offsets,ob_offsets
def make_output(doc2pred_ner_doc,simple_thre,complex_thre,text_data,pred_info):
    for ind,info in tqdm(enumerate(doc2pred_ner_doc)):
        rels={}
        for idx in (info):
            rels[pred_doc_rels[idx]]=idx
        for idx in info:
            token_label_pred=text_preds[idx]
            label_pred=pred_doc_rels[idx]
            text=text_data[ind]['text']
            if label_pred in special_pass_idx:
                continue
            elif label_pred not in special_major_idx:
                subs,obs,sub_offsets,ob_offsets=find_subs_obs(token_label_pred,text)
                if (len(subs)*len(obs)<simple_thre) or len(subs)==1 or len(obs)==1:
                    for i,sub in enumerate(subs):
                        for j,ob in enumerate(obs):
                            if sub=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':sub}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            pred_info[ind]['spo_list'].append(rel_dict)
                else:
                    for j,ob in enumerate(obs):
    #                     sub_offset=sub_offsets[i]
    #                     min_dis,ob_index=100000,-1
                        ob_offset=ob_offsets[j]
                        min_dis,sub_index=100000,-1
                        for i,sub in enumerate(subs):
                            if sub=='' or ob=='':
                                continue
                            if abs(sub_offsets[i][0]-ob_offset[0])<min_dis:
                                sub_index=i
                                min_dis=abs(sub_offsets[i][0]-ob_offset[0])
                        if sub_index!=-1:
                            if subs[sub_index]=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':subs[sub_index]}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            pred_info[ind]['spo_list'].append(rel_dict)
            else:
                affilate_labels=special_affilate_idx[label_pred]
                subs,obs,sub_offsets,ob_offsets=find_subs_obs(token_label_pred,text)
                subs2id=dict([(sub,i) for i,sub in enumerate(subs)])
                subs_af=[[] for i in range(len(subs))]
                for f in affilate_labels:
                    if f in rels:
                        af_idx=rels[f]
                        af_token_label_pred=text_preds[af_idx]
                        af_text=text_data[ind]['text']
                        af_subs,af_obs,af_sub_offsets,af_ob_offsets=find_subs_obs(af_token_label_pred,af_text)
                        for sub in af_subs:
                            if sub in subs:
                                subs_af[subs2id[sub]].append([af_obs,f,af_ob_offsets])
                #现在只取第一个
                if len(subs)*len(obs)<complex_thre :
                    for i,sub in enumerate(subs):
                        sub_offset=sub_offsets[i]
                        for j,ob in enumerate(obs):
                            if sub=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':sub}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            if len(subs_af[i])==0:
                                    pred_info[ind]['spo_list'].append(rel_dict)
                            if len(subs_af[i])>=1:
                                ob_offset=ob_offsets[j]
                                for af_obs in subs_af[i]:
                                    af_label_pred=af_obs[1]
                                    af_min_dis,af_ob_index=100000,-1
                                    for k,(af_ob,af_offset) in enumerate(zip(af_obs[0],af_obs[2])):
                                        if af_ob=='':
                                            continue
                                        if abs(ob_offset[0]-af_offset[0])<af_min_dis:
                                            af_ob_index=k
                                            min_dis=abs(ob_offset[0]-af_offset[0])
                                    if af_ob_index!=-1:
                                        rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_obs[0][af_ob_index]})
                                        rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))

                                pred_info[ind]['spo_list'].append(rel_dict)
                else:
                    for i,sub in enumerate(subs):

                        sub_offset=sub_offsets[i]
                        min_dis,ob_index=100000,-1
                        for j,ob in enumerate(obs):
                            if sub=='' or ob=='':
                                continue
                            if abs(sub_offset[0]-ob_offsets[j][0])<min_dis:
                                ob_index=j
                                min_dis=abs(sub_offset[0]-ob_offsets[j][0])
                        if ob_index!=-1:
                            j=ob_index
                            if subs[i]=='' or obs[j]=='':
                                continue
                            rel_dict={'object':{'@value':obs[j]},'subject':subs[i]}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            if len(subs_af[i])==0:
                                    pred_info[ind]['spo_list'].append(rel_dict)
                            if len(subs_af[i])>=1:
                                for af_obs in subs_af[i]:
                                    af_label_pred=af_obs[1]
                                    rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))
                                    for af_ob in af_obs[0]:
                                        rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_ob})
                                        break
                                pred_info[ind]['spo_list'].append(rel_dict) 

In [4]:
text_data=[]
with open("./dataset/train_data/new_train_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        text_data.append(json.loads(d))
test_text_data=[]
with open("./dataset/test1_data/new_test1_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        test_text_data.append(json.loads(d))
test2_text_data=[]
with open("./dataset/test2_data/test2_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        test2_text_data.append(json.loads(d))
schema=[]
with open("./dataset/schema.json","r") as r:
    raw_schema=r.readlines()
    for d in raw_schema:
        schema.append(json.loads(d))
rels=set()
special_rels=set()
for e in schema:
        if len(e['object_type'].keys())==1:
            rels.add(e["predicate"])
        else:
            special_rels.add(e["predicate"])
            for key in e['object_type'].keys():
                rels.add(e['predicate']+"_"+key)
if not os.path.exists("./dataset/dict.pk"):
    special_rels=list(special_rels)
    id2rels=list(rels)
    rels2id=dict([(rel,idx) for idx,rel in enumerate(id2rels)])
    id2labels=["O","B-OBJ","I-OBJ","B-SUB","I-SUB","[category]","[SEP]","[CLS]","[PAD]"]
    label2ids=dict([ (label,idx) for idx,label in enumerate(id2labels)])
    pk.dump([special_rels,id2rels,rels2id,id2labels,label2ids],open("./dataset/dict.pk","wb"))
else:
    print("loading dict...")
    special_rels,id2rels,rels2id,id2labels,label2ids=pk.load(open("./dataset/dict.pk","rb"))
id2kglabels=['O','I']+['B-'+e+"-SUB" for e in id2rels]+['B-'+e+"-OB" for e in id2rels]
kglabels2id=dict([ (label,idx) for idx,label in enumerate(id2kglabels)])
new_id2labels=["O","B-OBJ","E-OBJ","B-SUB","E-SUB","[category]","[SEP]","[CLS]","[PAD]"]
new_label2ids=dict([ (label,idx) for idx,label in enumerate(new_id2labels)])
new_id2kglabels=['O']+['B-'+e+"-SUB" for e in id2rels]+['B-'+e+"-OB" for e in id2rels]+['E-'+e+"-SUB" for e in id2rels]+['E-'+e+"-OB" for e in id2rels]
new_kglabels2id=dict([ (label,idx) for idx,label in enumerate(new_id2kglabels)])
id2reltype=[[] for i in range(len(id2rels))]
for e in schema:
    if len(e['object_type'].keys())==1:
        rel=e["predicate"]
        ids=rels2id[rel]
        id2reltype[ids].append(e)
    else:
        for key in e['object_type'].keys():
            rel=e['predicate']+"_"+key
            ids=rels2id[rel]
            temp_e=copy.deepcopy(e)
            poped_keys=[]
            for k in temp_e['object_type'].keys():
                if k!=key:
                    poped_keys.append(k)
            for k in poped_keys:
                 temp_e['object_type'].pop(k)
            id2reltype[ids].append(temp_e)
id2schema=[e[0] for e in id2reltype]
id2rel_text=[[] for i in range(len(id2rels))]
id2rel_rawtext=[[] for i in range(len(id2rels))]
id2rel_token2text=[[] for i in range(len(id2rels))]
for rel in range(len(id2rels)):
    if id2rels[rel].split("_")[0] not in special_rels:
        cls_text=id2schema[rel]['subject_type']+","+id2schema[rel]['predicate']+","+id2schema[rel]['object_type']['@value']
    else:
        cls_text=id2schema[rel]['subject_type']+","+id2schema[rel]['predicate']+","+id2schema[rel]['object_type'][id2rels[rel].split("_")[1]]
    id2rel_text[rel]=tokenizer.tokenize(cls_text)
    id2rel_rawtext[rel]=cls_text
    id2rel_token2text[rel]=index_token(cls_text)[0]
    assert len(id2rel_token2text[rel])==len(id2rel_text[rel])
if not os.path.exists("./middle_data/rel_data_postag.pk"):
    jieba.enable_paddle() 
    jieba.enable_parallel(8)
    rel_cut_words=[]
    rel_cut_tags=[]
    for idx in tqdm(range(len(id2rel_rawtext))):
        words = pseg.lcut(id2rel_rawtext[idx],use_paddle=True) #jieba默认模式
        new_words=[w for w,t in words]
        new_tags=[t for w,t in words]
        rel_cut_words.append([idx,new_words])
        rel_cut_tags.append([idx,new_tags])
    rel_cut_words=[e[1] for e in sorted(rel_cut_words,key=lambda x:x[0])]
    rel_cut_tags=[e[1] for e in sorted(rel_cut_tags,key=lambda x:x[0])]
    rel_data_postag=[]
    for idx in tqdm(range(len(id2rel_rawtext))):
        assert len(id2rel_rawtext[idx].strip())==len("".join(rel_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in rel_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        pos_label=np.zeros(len(id2rel_rawtext[idx])).astype(np.int8)
        for i,(b,e) in enumerate(indexs):
            assert (id2rel_rawtext[idx][b]==rel_cut_words[idx][i][0] or _is_whitespace(id2rel_rawtext[idx].strip()[b])\
                   or _is_whitespace(rel_cut_words[idx][i][0])) \
                    and (id2rel_rawtext[idx].strip()[e]==rel_cut_words[idx][i][-1] \
                         or _is_whitespace(id2rel_rawtext[idx].strip()[e])  or _is_whitespace(rel_cut_words[idx][i][-1]))
            pos_label[b+1:e+1]=pos2id_BIO['I-'+rel_cut_tags[idx][i]]
            pos_label[b]=pos2id_BIO['B-'+rel_cut_tags[idx][i]]
        rel_data_postag.append(pos_label)
    pk.dump(rel_data_postag,open("./middle_data/rel_data_postag.pk","wb"))
else:
    print("loading...")
    rel_data_postag=pk.load(open("./middle_data/rel_data_postag.pk","rb"))
special_major_idx=[2,4,22,32,54]
special_affilate_idx=[[] for i in range(len(id2rels))]
special_affilate_idx[2]=[5]
special_affilate_idx[4]=[0]
special_affilate_idx[22]=[51]
special_affilate_idx[32]=[8]
special_affilate_idx[54]=[6,11,14]
special_pass_idx=[0,5,6,8,11,14,51]
padding_O=[0]*len(new_id2labels)
padding_O[0]=1
padding_category=[0]*len(new_id2labels)
padding_category[5]=1
padding_SEP=[0]*len(new_id2labels)
padding_SEP[6]=1
padding_CLS=[0]*len(new_id2labels)
padding_CLS[7]=1
padding_PAD=[0]*len(new_id2labels)
padding_PAD[8]=1 #for pointer ner

loading dict...
loading...


In [5]:
if not os.path.exists('./Tencent_ChineseEmbedding/ChineseEmbedding.bin'):
    file = './Tencent_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
    wv_from_text = KeyedVectors.load_word2vec_format(file, binary=False)
    wv_from_text.init_sims(replace=True)
    # 重新保存加载变量为二进制形式
    wv_from_text.save('./Tencent_ChineseEmbedding/ChineseEmbedding.bin')
else:
    print("loading...")
    wv_from_text = gensim.models.KeyedVectors.load('./Tencent_ChineseEmbedding/ChineseEmbedding.bin', mmap='r')

05/18/2020 23:06:11 - INFO - gensim.utils -   loading Word2VecKeyedVectors object from ./Tencent_ChineseEmbedding/ChineseEmbedding.bin


loading...


05/18/2020 23:06:48 - INFO - gensim.utils -   loading vectors from ./Tencent_ChineseEmbedding/ChineseEmbedding.bin.vectors.npy with mmap=r
05/18/2020 23:06:48 - INFO - gensim.utils -   setting ignored attribute vectors_norm to None
05/18/2020 23:06:48 - INFO - gensim.utils -   loaded ./Tencent_ChineseEmbedding/ChineseEmbedding.bin


In [6]:
if not os.path.exists("./Tencent_ChineseEmbedding/extra_embedding.npy"):
    word2vec_dims=wv_from_text.vectors.shape[-1]
    pad_vec=np.zeros([1,word2vec_dims],dtype=np.float32)
    other_token_vec=np.random.normal(scale=0.02,size=[3,word2vec_dims])
    extra_token_vec=np.concatenate([pad_vec,other_token_vec])
    np.save("./Tencent_ChineseEmbedding/extra_embedding.npy",extra_token_vec)
else:
    print("loading extra tokens vector file...")
    extra_token_vec=np.load("./Tencent_ChineseEmbedding/extra_embedding.npy")
word2vec=np.concatenate([extra_token_vec,wv_from_text.vectors])

loading extra tokens vector file...


In [7]:
if not os.path.exists('./Tencent_ChineseEmbedding/usr_dict.txt'):
    keys=[]
    for key in tqdm(wv_from_text.vocab.keys()):
        keys.append(key)
    with open("./Tencent_ChineseEmbedding/usr_dict.txt","w") as w:
        for key in tqdm(keys):
            w.write(key+"\n")
else:
    print("loading...")
    keys=[]
    with open("./Tencent_ChineseEmbedding/usr_dict.txt","r") as r:
        for key in tqdm(r.readlines()):
            keys.append(key[:-1])
import jieba
jieba.load_userdict("./Tencent_ChineseEmbedding/usr_dict.txt")
if not os.path.exists("./Tencent_ChineseEmbedding/word_id.pk"):
    word2id={}
    id2word=[[] for i in range(word2vec.shape[0])]
    for i,key in tqdm(enumerate(keys)):
        assert (word2vec[4+i]==wv_from_text[key]).all()
        word2id[key]=4+i
        id2word[4+i]=key
    id2word[0]=tokenizer.pad_token
    word2id[tokenizer.pad_token]=0
    id2word[1]=tokenizer.unk_token
    word2id[tokenizer.unk_token]=1
    id2word[2]=tokenizer.cls_token
    word2id[tokenizer.cls_token]=2
    id2word[3]=tokenizer.sep_token
    word2id[tokenizer.sep_token]=3
    pk.dump([word2id,id2word],open("./Tencent_ChineseEmbedding/word_id.pk","wb"),protocol = 4)
else:
    print("loading...")
    word2id,id2word=pk.load(open("./Tencent_ChineseEmbedding/word_id.pk","rb"))

loading...


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


Building prefix dict from the default dictionary ...
05/18/2020 23:07:05 - DEBUG - jieba -   Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
05/18/2020 23:07:05 - DEBUG - jieba -   Loading model from cache /tmp/jieba.cache


Loading model cost 0.954 seconds.
05/18/2020 23:07:06 - DEBUG - jieba -   Loading model cost 0.954 seconds.
Prefix dict has been built successfully.
05/18/2020 23:07:06 - DEBUG - jieba -   Prefix dict has been built successfully.


loading...


In [12]:
if not os.path.exists("./middle_data/train_wordvec.pk"):
    jieba.enable_parallel(8)
    train_cut_words=[]
    train_cut_vecs=[]
    for idx in tqdm(range(len(text_data))):
        word_list=jieba.lcut(text_data[idx]['text'],HMM=False,use_paddle=False)
        wvid_list=[ word2id.get(e,1)  for e in word_list ]
        train_cut_words.append([idx,word_list])
        train_cut_vecs.append([idx,wvid_list])
    train_cut_words=[e[1] for e in sorted(train_cut_words,key=lambda x:x[0])]
    train_cut_vecs=[e[1] for e in sorted(train_cut_vecs,key=lambda x:x[0])]
#     train_cut_wvs=[]
#     for e in tqdm(train_cut_vecs):
#         train_cut_wvs.append(np.array([word2vec[t] for t in e]))
    pk.dump([train_cut_words,train_cut_vecs],open("./middle_data/train_wordvec.pk","wb"))
else:
    print("loading...")
    train_cut_words,train_cut_vecs=pk.load(open("./middle_data/train_wordvec.pk","rb"))
if not os.path.exists("./middle_data/train_tokenvec.pk"):
    text_data_tokenvec=[]
    for idx in tqdm(range(len(text_data))):
        assert len(text_data[idx]['text'])==len("".join(train_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in train_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        vec_label=np.zeros((len(text_data[idx]['text']))).astype(np.int)
        for i,(b,e) in enumerate(indexs):
            assert (text_data[idx]['text'][b]==train_cut_words[idx][i][0] or _is_whitespace(text_data[idx]['text'][b])\
                   or _is_whitespace(train_cut_words[idx][i][0])) \
                    and (text_data[idx]['text'][e]==train_cut_words[idx][i][-1] \
                         or _is_whitespace(text_data[idx]['text'][e])  or _is_whitespace(train_cut_words[idx][i][-1]))
            vec_label[b:e+1]=train_cut_vecs[idx][i]
        text_data_tokenvec.append(vec_label)
    pk.dump(text_data_tokenvec,open("./middle_data/train_tokenvec.pk","wb"))
else:
    print("loading...")
    text_data_tokenvec=pk.load(open("./middle_data/train_tokenvec.pk","rb"))


loading...


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [17]:
if not os.path.exists("./middle_data/test_wordvec.pk"):
    jieba.enable_parallel(8)
    test_cut_words=[]
    test_cut_vecs=[]
    for idx in tqdm(range(len(test_text_data))):
        word_list=jieba.lcut(test_text_data[idx]['text'],HMM=False,use_paddle=False)
        wvid_list=[ word2id.get(e,1)  for e in word_list ]
        test_cut_words.append([idx,word_list])
        test_cut_vecs.append([idx,wvid_list])
    test_cut_words=[e[1] for e in sorted(test_cut_words,key=lambda x:x[0])]
    test_cut_vecs=[e[1] for e in sorted(test_cut_vecs,key=lambda x:x[0])]
    pk.dump([test_cut_words,test_cut_vecs],open("./middle_data/test_wordvec.pk","wb"))
else:
    print("loading...")
    test_cut_words,test_cut_vecs=pk.load(open("./middle_data/test_wordvec.pk","rb"))
if not os.path.exists("./middle_data/test_tokenvec.pk"):
    test_text_data_tokenvec=[]
    for idx in tqdm(range(len(test_text_data))):
        assert len(test_text_data[idx]['text'])==len("".join(test_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in test_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        vec_label=np.zeros((len(test_text_data[idx]['text']),)).astype(np.int)
        for i,(b,e) in enumerate(indexs):
            assert (test_text_data[idx]['text'][b]==test_cut_words[idx][i][0] or _is_whitespace(test_text_data[idx]['text'][b])\
                   or _is_whitespace(test_cut_words[idx][i][0])) \
                    and (test_text_data[idx]['text'][e]==test_cut_words[idx][i][-1] \
                         or _is_whitespace(test_text_data[idx]['text'][e])  or _is_whitespace(test_cut_words[idx][i][-1]))
            vec_label[b:e+1]=test_cut_vecs[idx][i]
        test_text_data_tokenvec.append(vec_label)
    pk.dump(test_text_data_tokenvec,open("./middle_data/test_tokenvec.pk","wb"))
else:
    print("loading...")
    test_text_data_tokenvec=pk.load(open("./middle_data/test_tokenvec.pk","rb"))

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [8]:
if not os.path.exists("./middle_data/test2_wordvec.pk"):
    jieba.enable_parallel(8)
    test2_cut_words=[]
    test2_cut_vecs=[]
    for idx in tqdm(range(len(test2_text_data))):
        word_list=jieba.lcut(test2_text_data[idx]['text'],HMM=False,use_paddle=False)
        wvid_list=[ word2id.get(e,1)  for e in word_list ]
        test2_cut_words.append([idx,word_list])
        test2_cut_vecs.append([idx,wvid_list])
    test2_cut_words=[e[1] for e in sorted(test2_cut_words,key=lambda x:x[0])]
    test2_cut_vecs=[e[1] for e in sorted(test2_cut_vecs,key=lambda x:x[0])]
    pk.dump([test2_cut_words,test2_cut_vecs],open("./middle_data/test2_wordvec.pk","wb"))
else:
    print("loading...")
    test2_cut_words,test2_cut_vecs=pk.load(open("./middle_data/test2_wordvec.pk","rb"))
if not os.path.exists("./middle_data/test2_tokenvec.pk"):
    test2_text_data_tokenvec=[]
    for idx in tqdm(range(len(test2_text_data))):
        assert len(test2_text_data[idx]['text'])==len("".join(test2_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in test2_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        vec_label=np.zeros((len(test2_text_data[idx]['text']),)).astype(np.int)
        for i,(b,e) in enumerate(indexs):
            assert (test2_text_data[idx]['text'][b]==test2_cut_words[idx][i][0] or _is_whitespace(test2_text_data[idx]['text'][b])\
                   or _is_whitespace(test2_cut_words[idx][i][0])) \
                    and (test2_text_data[idx]['text'][e]==test2_cut_words[idx][i][-1] \
                         or _is_whitespace(test2_text_data[idx]['text'][e])  or _is_whitespace(test2_cut_words[idx][i][-1]))
            vec_label[b:e+1]=test2_cut_vecs[idx][i]
        test2_text_data_tokenvec.append(vec_label)
    pk.dump(test2_text_data_tokenvec,open("./middle_data/test2_tokenvec.pk","wb"))
else:
    print("loading...")
    test2_text_data_tokenvec=pk.load(open("./middle_data/test2_tokenvec.pk","rb"))


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
